In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split

data=pd.read_csv("C:\\Users\\jaban\\Downloads\\OLIST\\output\\final_merged_dataset.csv")

data=data.drop("Unnamed: 0",axis=1)

X = data.copy()

categories_col = ['books_cds_media',
                  'fashion_clothing_accessories',
                  'flowers_gifts',
                  'groceries_food_drink',
                  'health_beauty',
                  'home_furniture',
                  'other',
                  'sport',
                  'technology',
                  'toys_baby']

X = X.drop(categories_col, axis=1)
X=X.drop("customer_unique_id",axis=1)
X=X.drop("index",axis=1)

# # numerical_features = list(data.select_dtypes(include=['int64','float64', 'uint8']).columns)
# numerical_features = list(X.select_dtypes(include=['int64','float64', 'uint8']).columns)

# y = X[["mean_review_score","freight_ratio"]]

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )
customers = X.copy()



In [2]:
X.columns

Index(['recency', 'frequency', 'monetary', 'nb_orders', 'total_spend',
       'count_payment_sequential', 'mean_payment_installments',
       'mean_review_score', 'mean_delivery_days', 'average_orders_weight',
       'favorite_sale_month', 'total_items', 'order_mean_delay',
       'freight_ratio'],
      dtype='object')

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scalar_vars = customers[customers.select_dtypes(exclude=["object", "category"]).columns]

scaled_customers = scaler.fit_transform(scalar_vars)

scaled_customers

array([[-0.83472867, -0.1607334 , -0.03459677, ..., -0.29766964,
        -0.78890591, -1.02160733],
       [-0.81514195, -0.1607334 , -0.61591313, ..., -0.29766964,
        -0.76954777,  0.74296037],
       [ 1.94658602, -0.1607334 , -0.3535352 , ..., -0.29766964,
         1.95995075, -0.05911586],
       ...,
       [ 2.14898215, -0.1607334 , -0.24408014, ..., -0.29766964,
         2.15998492, -0.05911586],
       [-0.78249741, -0.1607334 , -0.11262933, ..., -0.29766964,
        -0.73728419, -0.54036159],
       [ 1.60055391, -0.1607334 , -0.41643259, ..., -0.29766964,
         1.61795685, -0.05911586]])

In [8]:
from sklearn.neighbors import NearestNeighbors

nearest_neighbors = NearestNeighbors(n_neighbors=26).fit(scaled_customers)
distances, idx = nearest_neighbors.kneighbors(scaled_customers)
distances = np.sort(distances, axis=0)
distances = distances[:, 1]
fig = px.line(None, y=distances, title="Evaluation of epsilon")
fig.show()

In [9]:
len(X)

94488

In [11]:
from sklearn import metrics
import numpy as np

# Compute DBSCAN
report = []
for eps_value in np.arange(3, 6, 0.2):
    dbscan = DBSCAN(eps=eps_value, min_samples=26, n_jobs=-1).fit(scaled_customers)
    labels = dbscan.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    score = metrics.silhouette_score(scaled_customers, labels)
    report.append(
        {
            "eps": eps_value,
            "n_clusters": n_clusters_,
            "noise_points": n_noise_,
            "silhouette_score": score,
        }
    )

df_report = pd.DataFrame(report)
df_report.plot(
    title="Exploring eps value for an optimal silhouette score",
    x="eps",
    y="silhouette_score",
)

df_report.head(15)

MemoryError: 

In [ ]:
# Compute DBSCAN
dbscan = DBSCAN(eps=5, min_samples=26).fit(scaled_customers)
labels = dbscan.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

print(
    "Silhouette Coefficient: %0.3f" % metrics.silhouette_score(scaled_customers, labels)
)

In [ ]:
customers["cluster"] = labels
customers.cluster = customers.cluster.astype("category")

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne = TSNE(n_components=2, learning_rate="auto", init="random", n_jobs=-1)
projected_customers = tsne.fit_transform(scaled_customers)

fig = px.scatter(
    projected_customers,
    x=0,
    y=1,
    color=customers.cluster,
    labels={"color": "cluster"},
    opacity=0.8,
)
fig.show()

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne = TSNE(n_components=2, learning_rate="auto", init="random", n_jobs=-1)
projected_customers = tsne.fit_transform(scaled_customers)

fig = px.scatter(
    projected_customers,
    x=0,
    y=1,
    color=customers.cluster,
    labels={"color": "cluster"},
    opacity=0.8,
)
fig.show()